In [1]:
from moleculekit.molecule import Molecule
import os

testdir = "data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # Reading the system topology
mol.read(os.path.join(testdir, "input.coor"))  # Reading the initial simulation coordinates
mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

In [3]:
from torchmd.forcefields.forcefield import ForceField



ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))

In [39]:
import jax
from math import sqrt
import jax.numpy as nnp
import numpy as np

class Parameters:
    def __init__(
        self,
        ff,
        mol,
        terms=None,
        precision=float,
        device="cpu",
    ):
        self.A = None
        self.B = None
        self.bonds = None
        self.bond_params = None
        self.charges = None
        self.masses = None
        self.mapped_atom_types = None
        self.angles = None
        self.angle_params = None
        self.dihedrals = None
        self.dihedral_params = None
        self.idx14 = None
        self.nonbonded_14_params = None
        self.impropers = None
        self.improper_params = None

        self.natoms = mol.numAtoms
        if terms is None:
            terms = ("bonds", "angles", "dihedrals", "impropers", "1-4", "lj")
        terms = [term.lower() for term in terms]
        self.build_parameters(ff, mol, terms)
        # self.precision_(precision)
        # self.to_(device)

        self.device = device

    # def to_(self, device):
    #     self.charges = self.charges.to(device)
    #     self.masses = self.masses.to(device)
    #     if self.A is not None:
    #         self.A = self.A.to(device)
    #     if self.B is not None:
    #         self.B = self.B.to(device)
    #     if self.bonds is not None:
    #         self.bonds = self.bonds.to(device)
    #         self.bond_params = self.bond_params.to(device)
    #     if self.angles is not None:
    #         self.angles = self.angles.to(device)
    #         self.angle_params = self.angle_params.to(device)
    #     if self.dihedrals is not None:
    #         self.dihedrals = self.dihedrals.to(device)
    #         for j in range(len(self.dihedral_params)):
    #             termparams = self.dihedral_params[j]
    #             termparams["idx"] = termparams["idx"].to(device)
    #             termparams["params"] = termparams["params"].to(device)
    #     if self.idx14 is not None:
    #         self.idx14 = self.idx14.to(device)
    #         self.nonbonded_14_params = self.nonbonded_14_params.to(device)
    #     if self.impropers is not None:
    #         self.impropers = self.impropers.to(device)
    #         termparams = self.improper_params[0]
    #         termparams["idx"] = termparams["idx"].to(device)
    #         termparams["params"] = termparams["params"].to(device)
    #     if self.mapped_atom_types is not None:
    #         self.mapped_atom_types = self.mapped_atom_types.to(device)
    #     self.device = device

    # def precision_(self, precision):
    #     self.charges = self.charges.type(precision)
    #     self.masses = self.masses.type(precision)
    #     if self.A is not None:
    #         self.A = self.A.type(precision)
    #     if self.B is not None:
    #         self.B = self.B.type(precision)
    #     if self.bonds is not None:
    #         self.bond_params = self.bond_params.type(precision)
    #     if self.angles is not None:
    #         self.angle_params = self.angle_params.type(precision)
    #     if self.dihedrals is not None:
    #         for j in range(len(self.dihedral_params)):
    #             termparams = self.dihedral_params[j]
    #             termparams["params"] = termparams["params"].type(precision)
    #     if self.idx14 is not None:
    #         self.nonbonded_14_params = self.nonbonded_14_params.type(precision)
    #     if self.impropers is not None:
    #         termparams = self.improper_params[0]
    #         termparams["params"] = termparams["params"].type(precision)

    def get_exclusions(self, types=("bonds", "angles", "1-4"), fullarray=False):
        exclusions = []
        if self.bonds is not None and "bonds" in types:
            exclusions += self.bonds.tolist()
        if self.angles is not None and "angles" in types:
            npangles = self.angles
            exclusions += npangles[:, [0, 2]].tolist()
        if self.dihedrals is not None and "1-4" in types:
            # These exclusions will be covered by nonbonded_14_params
            npdihedrals = self.dihedrals
            exclusions += npdihedrals[:, [0, 3]].tolist()
        if fullarray:
            initial_fullmat = nnp.full((self.natoms, self.natoms), False, dtype=bool)
            if len(exclusions):
                exclusions = nnp.array(exclusions)
                fullmat = initial_fullmat.at[exclusions[:, 0], exclusions[:, 1]].set(True)
                fullmat = initial_fullmat.at[exclusions[:, 1], exclusions[:, 0]].set(True)
                exclusions = fullmat
        return exclusions

    def build_parameters(self, ff, mol, terms):
        print('foo')
        uqatomtypes, indexes = np.unique(mol.atomtype, return_inverse=True)
        print('baz')
        self.mapped_atom_types = np.array(indexes)
        print('bar')
        # change point
        self.charges = np.array(mol.charge.astype(np.float64))
        self.masses = self.make_masses(ff, mol.atomtype)
        if "lj" in terms or "LJ" in terms:
            self.A, self.B = self.make_lj(ff, uqatomtypes)
        if "bonds" in terms and len(mol.bonds):
            uqbonds = np.unique([sorted(bb) for bb in mol.bonds], axis=0)
            self.bonds = np.array(uqbonds.astype(np.int64))
            self.bond_params = self.make_bonds(ff, uqatomtypes[indexes[uqbonds]])
        if "angles" in terms and len(mol.angles):
            uqangles = np.unique(
                [ang if ang[0] < ang[2] else ang[::-1] for ang in mol.angles], axis=0
            )
            self.angles = np.array(uqangles.astype(np.int64))
            self.angle_params = self.make_angles(ff, uqatomtypes[indexes[uqangles]])
        if "dihedrals" in terms and len(mol.dihedrals):
            uqdihedrals = np.unique(
                [dih if dih[0] < dih[3] else dih[::-1] for dih in mol.dihedrals], axis=0
            )
            self.dihedrals = np.array(uqdihedrals.astype(np.int64))
            self.dihedral_params = self.make_dihedrals(
                ff, uqatomtypes[indexes[uqdihedrals]]
            )
        if "1-4" in terms and len(mol.dihedrals):
            # Keep only dihedrals whos 1/4 atoms are not in bond+angle exclusions
            exclusions = self.get_exclusions(types=("bonds", "angles"), fullarray=True)
            keep = ~exclusions[uqdihedrals[:, 0], uqdihedrals[:, 3]]
            dih14 = uqdihedrals[keep, :]
            if len(dih14):
                # Remove duplicates (can occur if 1,4 atoms were same and 2,3 differed)
                uq14idx = np.unique(dih14[:, [0, 3]], axis=0, return_index=True)[1]
                dih14 = dih14[uq14idx]
                self.idx14 = np.array(dih14[:, [0, 3]].astype(np.int64))
                self.nonbonded_14_params = self.make_14(ff, uqatomtypes[indexes[dih14]])
        if "impropers" in terms and len(mol.impropers):
            uqimpropers = np.unique(mol.impropers, axis=0)
            # uqimpropers = self._unique_impropers(mol.impropers, mol.bonds)
            self.impropers = np.array(uqimpropers.astype(np.int64))
            self.improper_params = self.make_impropers(
                ff, uqatomtypes, indexes, uqimpropers, uqbonds
            )

    # def make_charges(self, ff, atomtypes):
    #     return np.array([ff.get_charge(at) for at in atomtypes])

    def make_masses(self, ff, atomtypes):
        masses = np.array([ff.get_mass(at) for at in atomtypes])
        masses = np.expand_dims(masses,1)  # natoms,1
        return masses

    def make_lj(self, ff, uqatomtypes):
        sigma = []
        epsilon = []
        for at in uqatomtypes:
            ss, ee = ff.get_LJ(at)
            sigma.append(ss)
            epsilon.append(ee)

        sigma = np.array(sigma, dtype=np.float64)
        epsilon = np.array(epsilon, dtype=np.float64)

        A, B = calculate_AB(sigma, epsilon)
        A = np.array(A)
        B = np.array(B)
        return A, B

    def make_bonds(self, ff, uqbondatomtypes):
        return np.array([ff.get_bond(*at) for at in uqbondatomtypes])

    def make_angles(self, ff, uqangleatomtypes):
        return np.array([ff.get_angle(*at) for at in uqangleatomtypes])

    def make_dihedrals(self, ff, uqdihedralatomtypes):
        from collections import defaultdict

        dihedrals = defaultdict(lambda: {"idx": [], "params": []})

        for i, at in enumerate(uqdihedralatomtypes):
            terms = ff.get_dihedral(*at)
            for j, term in enumerate(terms):
                dihedrals[j]["idx"].append(i)
                dihedrals[j]["params"].append(term)

        maxterms = max(dihedrals.keys()) + 1
        newdihedrals = []
        for j in range(maxterms):
            dihedrals[j]["idx"] = np.array(dihedrals[j]["idx"])
            dihedrals[j]["params"] = np.array(dihedrals[j]["params"])
            newdihedrals.append(dihedrals[j])

        return newdihedrals

    def make_impropers(self, ff, uqatomtypes, indexes, uqimpropers, bonds):
        impropers = {"idx": [], "params": []}
        graph = improper_graph(uqimpropers, bonds)

        for i, impr in enumerate(uqimpropers):
            at = uqatomtypes[indexes[impr]]
            try:
                params = ff.get_improper(*at)
            except:
                center = detect_improper_center(impr, graph)
                notcenter = sorted(np.setdiff1d(impr, center))
                order = [notcenter[0], notcenter[1], center, notcenter[2]]
                at = uqatomtypes[indexes[order]]
                params = ff.get_improper(*at)

            impropers["idx"].append(i)
            impropers["params"].append(params)

        impropers["idx"] = np.array(impropers["idx"])
        impropers["params"] = np.array(impropers["params"])
        return [impropers]

    def make_14(self, ff, uq14atomtypes):
        nonbonded_14_params = []
        for uqdih in uq14atomtypes:
            scnb, scee, lj1_s14, lj1_e14, lj4_s14, lj4_e14 = ff.get_14(*uqdih)
            # Lorentz - Berthelot combination rule
            sig = 0.5 * (lj1_s14 + lj4_s14)
            eps = sqrt(lj1_e14 * lj4_e14)
            s6 = sig**6
            s12 = s6 * s6
            A = eps * 4 * s12
            B = eps * 4 * s6
            nonbonded_14_params.append([A, B, scnb, scee])
        return np.array(nonbonded_14_params)


def calculate_AB(sigma, epsilon):
    # Lorentz - Berthelot combination rule
    sigma_table = 0.5 * (sigma + sigma[:, None])
    eps_table = np.sqrt(epsilon * epsilon[:, None])
    sigma_table_6 = sigma_table**6
    sigma_table_12 = sigma_table_6 * sigma_table_6
    A = eps_table * 4 * sigma_table_12
    B = eps_table * 4 * sigma_table_6
    del sigma_table_12, sigma_table_6, eps_table, sigma_table
    return A, B


def detect_improper_center(indexes, graph):
    for i in indexes:
        if len(np.intersect1d(list(graph.neighbors(i)), indexes)) == 3:
            return i


def improper_graph(impropers, bonds):
    import networkx as nx

    g = nx.Graph()
    g.add_nodes_from(np.unique(impropers))
    g.add_edges_from([tuple(b) for b in bonds])
    return g


In [40]:
parameters = Parameters(ff, mol, precision=float, device='cpu')

foo
baz
bar


In [188]:
def set_positions(nreplicas, pos):
    if pos.shape[1] != 3:
        raise RuntimeError(
            "Positions shape must be (natoms, 3, 1) or (natoms, 3, nreplicas)"
        )

    atom_pos = np.transpose(pos, (2, 0, 1))
    if nreplicas > 1 and atom_pos.shape[0] != nreplicas:
        atom_pos = np.repeat(atom_pos[0][None, :], nreplicas, axis=0)

    # self.pos[:] = torch.tensor(
    #     atom_pos, dtype=self.pos.dtype, device=self.pos.device
    # )
    return atom_pos

# def set_velocities(vel):
#     if vel.shape != (self.nreplicas, self.natoms, 3):
#         raise RuntimeError("Velocities shape must be (nreplicas, natoms, 3)")
#     self.vel[:] = vel.clone().detach().type(self.vel.dtype).to(self.vel.device)

def set_box(nreplicas, box):
    if box.ndim == 1:
        if len(box) != 3:
            raise RuntimeError("Box must have at least 3 elements")
        box = box[:, None]

    if box.shape[0] != 3:
        raise RuntimeError("Box shape must be (3, 1) or (3, nreplicas)")

    box = np.swapaxes(box, 1, 0)

    if nreplicas > 1 and box.shape[0] != nreplicas:
        box = np.repeat(box[0][None, :], nreplicas, axis=0)

    new_box = np.zeros((nreplicas, 3, 3))
    for r in range(box.shape[0]):
        new_box[r][np.array(np.eye(3), dtype=bool)] = np.array(
            box[r], dtype=box.dtype
        )
    return new_box

In [171]:
from scipy import constants as const
# import torch
import numpy as np
import numpy.linalg as npl 
import jax
from math import pi


class Forces:
    """
    Parameters
    ----------
    cutoff : float
        If set to a value it will only calculate LJ, electrostatics and bond energies for atoms which are closer
        than the threshold
    rfa : bool
        Use with `cutoff` to enable the reaction field approximation for scaling of the electrostatics up to the cutoff.
        Uses the value of `solventDielectric` to model everything beyond the cutoff distance as solvent with uniform
        dielectric.
    solventDielectric : float
        Used together with `cutoff` and `rfa`
    """

    # 1-4 is nonbonded but we put it currently in bonded to not calculate all distances
    bonded = ["bonds", "angles", "dihedrals", "impropers", "1-4"]
    nonbonded = ["electrostatics", "lj", "repulsion", "repulsioncg"]
    terms = bonded + nonbonded

    def __init__(
        self,
        parameters,
        terms=None,
        external=None,
        cutoff=None,
        rfa=False,
        solventDielectric=78.5,
        switch_dist=None,
        exclusions=("bonds", "angles", "1-4"),
    ):
        self.par = parameters
        if terms is None:
            raise RuntimeError(
                'Set force terms or leave empty brackets [].\nAvailable options: "bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj", "repulsion", "repulsioncg".'
            )

        self.energies = [ene.lower() for ene in terms]
        for et in self.energies:
            if et not in Forces.terms:
                raise ValueError(f"Force term {et} is not implemented.")

        if "1-4" in self.energies and "dihedrals" not in self.energies:
            raise RuntimeError(
                "You cannot enable 1-4 interactions without enabling dihedrals"
            )

        self.natoms = len(parameters.masses)
        self.require_distances = any(f in self.nonbonded for f in self.energies)
        self.ava_idx = (
            self._make_indeces(
                self.natoms, parameters.get_exclusions(exclusions), parameters.device
            )
            if self.require_distances
            else None
        )
        self.external = external
        self.cutoff = cutoff
        self.rfa = rfa
        self.solventDielectric = solventDielectric
        self.switch_dist = switch_dist

    def _filter_by_cutoff(self, dist, arrays):
        under_cutoff = dist <= self.cutoff
        indexedarrays = []
        for arr in arrays:
            indexedarrays.append(arr[under_cutoff])
        return indexedarrays

    def compute(self, pos, box, forces, returnDetails=False, explicit_forces=True):
        if not explicit_forces and not pos.requires_grad:
            raise RuntimeError(
                "The positions passed don't require gradients. Please use pos.detach().requires_grad_(True) before passing."
            )

        nsystems = pos.shape[0]
        if np.any(np.isnan(pos)):
            raise RuntimeError("Found NaN coordinates.")

        pot = []
        for i in range(nsystems):
            pp = {
                v: np.zeros((1,))
                for v in self.energies
            }
            pp["external"] = np.zeros((1, ))
            pot.append(pp)

        # todo: change point
        # forces.zero_()
        for i in range(nsystems):
            spos = pos[i]
            sbox = box[i][np.array(np.eye(3), dtype=bool)]  # Use only the diagonal
            print("SBOX", sbox)
            # Bonded terms
            # TODO: We are for sure doing duplicate distance calculations here!
            if "bonds" in self.energies and self.par.bonds is not None:
                bond_dist, bond_unitvec, _ = calculate_distances(
                    spos, self.par.bonds, sbox
                )
                pairs = self.par.bonds
                bond_params = self.par.bond_params
                if self.cutoff is not None:
                    (
                        bond_dist,
                        bond_unitvec,
                        pairs,
                        bond_params,
                    ) = self._filter_by_cutoff(
                        bond_dist, (bond_dist, bond_unitvec, pairs, bond_params)
                    )
                E, force_coeff = evaluate_bonds(bond_dist, bond_params, explicit_forces)

                pot[i]["bonds"] += E.sum()
                if explicit_forces:
                    forcevec = bond_unitvec * force_coeff[:, None]
                    np.add.at(forces[i], pairs[:, 0], -forcevec)
                    np.add.at(forces[i], pairs[:, 1], -forcevec)
                    # forces[i].index_add_(0, pairs[:, 0], -forcevec)
                    # forces[i].index_add_(0, pairs[:, 1], forcevec)

            if "angles" in self.energies and self.par.angles is not None:
                _, _, r21 = calculate_distances(spos, self.par.angles[:, [0, 1]], sbox)
                _, _, r23 = calculate_distances(spos, self.par.angles[:, [2, 1]], sbox)
                E, angle_forces = evaluate_angles(
                    r21, r23, self.par.angle_params, explicit_forces
                )

                pot[i]["angles"] += E.sum()
                if explicit_forces:

                
                    
                    np.add.at(forces[i], self.par.angles[:, 0], angle_forces[0])
                    np.add.at(forces[i], self.par.angles[:, 1], angle_forces[1])
                    np.add.at(forces[i], self.par.angles[:, 2], angle_forces[2])


                    # forces[i].index_add_(0, self.par.angles[:, 0], angle_forces[0])
                    # forces[i].index_add_(0, self.par.angles[:, 1], angle_forces[1])
                    # forces[i].index_add_(0, self.par.angles[:, 2], angle_forces[2])

            if "dihedrals" in self.energies and self.par.dihedrals is not None:
                _, _, r12 = calculate_distances(
                    spos, self.par.dihedrals[:, [0, 1]], sbox
                )
                _, _, r23 = calculate_distances(
                    spos, self.par.dihedrals[:, [1, 2]], sbox
                )
                _, _, r34 = calculate_distances(
                    spos, self.par.dihedrals[:, [2, 3]], sbox
                )
                E, dihedral_forces = evaluate_torsion(
                    r12, r23, r34, self.par.dihedral_params, explicit_forces
                )

                pot[i]["dihedrals"] += E.sum()
                if explicit_forces:

                    #  
                    np.add.at(forces[i], self.par.dihedrals[:, 0], dihedral_forces[0])
                    np.add.at(forces[i], self.par.dihedrals[:, 1], dihedral_forces[1])
                    np.add.at(forces[i], self.par.dihedrals[:, 2], dihedral_forces[2])
                    np.add.at(forces[i], self.par.dihedrals[:, 3], dihedral_forces[3])

                    # forces[i].index_add_(
                    #     0, self.par.dihedrals[:, 0], dihedral_forces[0]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.dihedrals[:, 1], dihedral_forces[1]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.dihedrals[:, 2], dihedral_forces[2]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.dihedrals[:, 3], dihedral_forces[3]
                    # )

                    pass

            if "1-4" in self.energies and self.par.idx14 is not None:
                nb_dist, nb_unitvec, _ = calculate_distances(spos, self.par.idx14, sbox)

                nonbonded_14_params = self.par.nonbonded_14_params
                idx14 = self.par.idx14
                # if self.cutoff is not None:
                #     (
                #         nb_dist,
                #         nb_unitvec,
                #         nonbonded_14_params,
                #         idx14,
                #     ) = self._filter_by_cutoff(
                #         nb_dist,
                #         (
                #             nb_dist,
                #             nb_unitvec,
                #             self.par.nonbonded_14_params,
                #             self.par.idx14,
                #         ),
                #     )

                aa = nonbonded_14_params[:, 0]
                bb = nonbonded_14_params[:, 1]
                scnb = nonbonded_14_params[:, 2]
                scee = nonbonded_14_params[:, 3]

                if "lj" in self.energies:
                    E, force_coeff = evaluate_LJ_internal(
                        nb_dist, aa, bb, scnb, None, None, explicit_forces
                    )
                    pot[i]["lj"] += E.sum()
                    if explicit_forces:

                        forcevec = nb_unitvec * force_coeff[:, None]

                        np.add.at(forces[i], idx14[:, 0], -forcevec)
                        np.add.at(forces[i], idx14[:, 1], -forcevec)
                        
                        # forces[i].index_add_(0, idx14[:, 0], -forcevec)
                        # forces[i].index_add_(0, idx14[:, 1], forcevec)
                if "electrostatics" in self.energies:
                    E, force_coeff = evaluate_electrostatics(
                        nb_dist,
                        idx14,
                        self.par.charges,
                        scee,
                        cutoff=None,
                        rfa=False,
                        solventDielectric=self.solventDielectric,
                        explicit_forces=explicit_forces,
                    )
                    pot[i]["electrostatics"] += E.sum()
                    if explicit_forces:
                        forcevec = nb_unitvec * force_coeff[:, None]
                        np.add.at(forces[i], idx14[:, 0], -forcevec)
                        np.add.at(forces[i], idx14[:, 1], -forcevec)
                        # forces[i].index_add_(0, idx14[:, 0], -forcevec)
                        # forces[i].index_add_(0, idx14[:, 1], forcevec)

            if "impropers" in self.energies and self.par.impropers is not None:
                _, _, r12 = calculate_distances(
                    spos, self.par.impropers[:, [0, 1]], sbox
                )
                _, _, r23 = calculate_distances(
                    spos, self.par.impropers[:, [1, 2]], sbox
                )
                _, _, r34 = calculate_distances(
                    spos, self.par.impropers[:, [2, 3]], sbox
                )
                E, improper_forces = evaluate_torsion(
                    r12, r23, r34, self.par.improper_params, explicit_forces
                )

                pot[i]["impropers"] += E.sum()
                if explicit_forces:
                     

                    np.add.at(forces[i], self.par.impropers[:, 0], improper_forces[0])
                    np.add.at(forces[i], self.par.impropers[:, 1], improper_forces[1])
                    np.add.at(forces[i], self.par.impropers[:, 2], improper_forces[2])
                    np.add.at(forces[i], self.par.impropers[:, 3], improper_forces[3])
                    # forces[i].index_add_(
                    #     0, self.par.impropers[:, 0], improper_forces[0]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.impropers[:, 1], improper_forces[1]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.impropers[:, 2], improper_forces[2]
                    # )
                    # forces[i].index_add_(
                    #     0, self.par.impropers[:, 3], improper_forces[3]
                    # )
                    pass

            # Non-bonded terms
            if self.require_distances and len(self.ava_idx):
                # Lazy mode: Do all vs all distances
                # TODO: These distance calculations are fucked once we do neighbourlists since they will vary per system!!!!
                nb_dist, nb_unitvec, _ = calculate_distances(spos, self.ava_idx, sbox)
                ava_idx = self.ava_idx
                if self.cutoff is not None:
                    nb_dist, nb_unitvec, ava_idx = self._filter_by_cutoff(
                        nb_dist, (nb_dist, nb_unitvec, ava_idx)
                    )

                for v in self.energies:
                    if v == "electrostatics":
                        E, force_coeff = evaluate_electrostatics(
                            nb_dist,
                            ava_idx,
                            self.par.charges,
                            cutoff=self.cutoff,
                            rfa=self.rfa,
                            solventDielectric=self.solventDielectric,
                            explicit_forces=explicit_forces,
                        )
                        pot[i][v] += E.sum()
                    elif v == "lj":
                        E, force_coeff = evaluate_LJ(
                            nb_dist,
                            ava_idx,
                            self.par.mapped_atom_types,
                            self.par.A,
                            self.par.B,
                            self.switch_dist,
                            self.cutoff,
                            explicit_forces,
                        )
                        pot[i][v] += E.sum()
                    elif v == "repulsion":
                        E, force_coeff = evaluate_repulsion(
                            nb_dist,
                            ava_idx,
                            self.par.mapped_atom_types,
                            self.par.A,
                            explicit_forces,
                        )
                        pot[i][v] += E.sum()
                    elif v == "repulsioncg":
                        E, force_coeff = evaluate_repulsion_CG(
                            nb_dist,
                            ava_idx,
                            self.par.mapped_atom_types,
                            self.par.B,
                            explicit_forces,
                        )
                        pot[i][v] += E.sum()
                    else:
                        continue

                    if explicit_forces:

                         
                        forcevec = nb_unitvec * force_coeff[:, None]
                        
                        np.add.at(forces[i], ava_idx[:, 0], -forcevec)
                        np.add.at(forces[i], ava_idx[:, 1], -forcevec)
                        # forces[i].index_add_(0, ava_idx[:, 0], -forcevec)
                        # forces[i].index_add_(0, ava_idx[:, 1], forcevec)

        if self.external:
            ext_ene, ext_force = self.external.calculate(pos, box)
            for s in range(nsystems):
                pot[s]["external"] += ext_ene[s]
            if explicit_forces:
                forces += ext_force

        if not explicit_forces:
            enesum = np.zeros(1, dtype=pos.dtype)
            for i in range(nsystems):
                for ene in pot[i]:
                    if pot[i][ene].requires_grad:
                        enesum += pot[i][ene]
            forces[:] = -jax.grad(
                enesum, pos, only_inputs=True, retain_graph=True
            )[0]
            if returnDetails:
                return pot
            else:
                return [np.sum(np.concatenate(list(pp.values()))) for pp in pot]

        if returnDetails:
            return [{k: v[0] for k, v in pp.items()} for pp in pot]
        else:
            return [np.sum([v[0] for _, v in pp.items()]) for pp in pot]

    def _make_indeces(self, natoms, excludepairs, device):
        fullmat = np.full((natoms, natoms), True, dtype=bool)
        if len(excludepairs):
            excludepairs = np.array(excludepairs)
            fullmat[excludepairs[:, 0], excludepairs[:, 1]] = False
            fullmat[excludepairs[:, 1], excludepairs[:, 0]] = False
        fullmat = np.triu(fullmat, +1)
        allvsall_indeces = np.vstack(np.where(fullmat)).T
        ava_idx = np.array(allvsall_indeces)
        return ava_idx


def wrap_dist(dist, box):
    if box is None or np.all(box == 0):
        wdist = dist
    else:
        wdist = dist - np.expand_dims(box,0) * np.round(dist / np.expand_dims(box,0))
    return wdist


def calculate_distances(atom_pos, atom_idx, box):
    
    direction_vec = wrap_dist(atom_pos[atom_idx[:, 0]] - atom_pos[atom_idx[:, 1]], box)
    # print(direction_vec, "DIR VEC")
    # print(atom_pos, "ATOM POS")
    # print(atom_idx, "ATOM IDX")
    dist = npl.norm(direction_vec, axis=1)
    # print(dist, 'DIST')
    direction_unitvec = direction_vec / np.expand_dims(dist,1)
    return dist, direction_unitvec, direction_vec


ELEC_FACTOR = 1 / (4 * const.pi * const.epsilon_0)  # Coulomb's constant
ELEC_FACTOR *= const.elementary_charge**2  # Convert elementary charges to Coulombs
ELEC_FACTOR /= const.angstrom  # Convert Angstroms to meters
ELEC_FACTOR *= const.Avogadro / (const.kilo * const.calorie)  # Convert J to kcal/mol


def evaluate_LJ(
    dist, pair_indeces, atom_types, A, B, switch_dist, cutoff, explicit_forces=True
):
    atomtype_indices = atom_types[pair_indeces]
    aa = A[atomtype_indices[:, 0], atomtype_indices[:, 1]]
    bb = B[atomtype_indices[:, 0], atomtype_indices[:, 1]]
    return evaluate_LJ_internal(dist, aa, bb, 1, switch_dist, cutoff, explicit_forces)


def evaluate_LJ_internal(
    dist, aa, bb, scale, switch_dist, cutoff, explicit_forces=True
):
    force = None

    rinv1 = 1 / dist
    rinv6 = rinv1**6
    rinv12 = rinv6 * rinv6

    pot = ((aa * rinv12) - (bb * rinv6)) / scale
    if explicit_forces:
        force = (-12 * aa * rinv12 + 6 * bb * rinv6) * rinv1 / scale

    # Switching function
    if switch_dist is not None and cutoff is not None:
        mask = dist > switch_dist
        t = (dist[mask] - switch_dist) / (cutoff - switch_dist)
        switch_val = 1 + t * t * t * (-10 + t * (15 - t * 6))
        if explicit_forces:
            switch_deriv = t * t * (-30 + t * (60 - t * 30)) / (cutoff - switch_dist)
            force[mask] = (
                switch_val * force[mask] + pot[mask] * switch_deriv / dist[mask]
            )
        pot[mask] = pot[mask] * switch_val

    return pot, force


def evaluate_repulsion(
    dist, pair_indeces, atom_types, A, scale=1, explicit_forces=True
):  # LJ without B
    force = None

    atomtype_indices = atom_types[pair_indeces]
    aa = A[atomtype_indices[:, 0], atomtype_indices[:, 1]]

    rinv1 = 1 / dist
    rinv6 = rinv1**6
    rinv12 = rinv6 * rinv6

    pot = (aa * rinv12) / scale
    if explicit_forces:
        force = (-12 * aa * rinv12) * rinv1 / scale
    return pot, force


def evaluate_repulsion_CG(
    dist, pair_indeces, atom_types, B, scale=1, explicit_forces=True
):  # Repulsion like from CGNet
    force = None

    atomtype_indices = atom_types[pair_indeces]
    coef = B[atomtype_indices[:, 0], atomtype_indices[:, 1]]

    rinv1 = 1 / dist
    rinv6 = rinv1**6

    pot = (coef * rinv6) / scale
    if explicit_forces:
        force = (-6 * coef * rinv6) * rinv1 / scale
    return pot, force


def evaluate_electrostatics(
    dist,
    pair_indeces,
    atom_charges,
    scale=1,
    cutoff=None,
    rfa=False,
    solventDielectric=78.5,
    explicit_forces=True,
):
    force = None
    if rfa:  # Reaction field approximation for electrostatics with cutoff
        # http://docs.openmm.org/latest/userguide/theory.html#coulomb-interaction-with-cutoff
        # Ilario G. Tironi, René Sperb, Paul E. Smith, and Wilfred F. van Gunsteren. A generalized reaction field method
        # for molecular dynamics simulations. Journal of Chemical Physics, 102(13):5451–5459, 1995.
        denom = (2 * solventDielectric) + 1
        krf = (1 / cutoff**3) * (solventDielectric - 1) / denom
        crf = (1 / cutoff) * (3 * solventDielectric) / denom
        common = (
            ELEC_FACTOR
            * atom_charges[pair_indeces[:, 0]]
            * atom_charges[pair_indeces[:, 1]]
            / scale
        )
        dist2 = dist**2
        pot = common * ((1 / dist) + krf * dist2 - crf)
        if explicit_forces:
            force = common * (2 * krf * dist - 1 / dist2)
    else:
        pot = (
            ELEC_FACTOR
            * atom_charges[pair_indeces[:, 0]]
            * atom_charges[pair_indeces[:, 1]]
            / dist
            / scale
        )
        if explicit_forces:
            force = -pot / dist
    return pot, force


def evaluate_bonds(dist, bond_params, explicit_forces=True):
    force = None

    k0 = bond_params[:, 0]
    d0 = bond_params[:, 1]
    x = dist - d0
    pot = k0 * (x**2)
    if explicit_forces:
        force = 2 * k0 * x
    return pot, force


def evaluate_angles(r21, r23, angle_params, explicit_forces=True):
    k0 = angle_params[:, 0]
    theta0 = angle_params[:, 1]

    dotprod = np.sum(r23 * r21, axis=1)
    norm23inv = 1 / npl.norm(r23, axis=1)
    norm21inv = 1 / npl.norm(r21, axis=1)

    cos_theta = dotprod * norm21inv * norm23inv
    cos_theta = np.clip(cos_theta, -1, 1)
    theta = np.arccos(cos_theta)

    delta_theta = theta - theta0
    pot = k0 * delta_theta * delta_theta

    force0, force1, force2 = None, None, None
    if explicit_forces:
        sin_theta = np.sqrt(1.0 - cos_theta * cos_theta)
        coef = np.zeros_like(sin_theta)
        nonzero = sin_theta != 0
        coef[nonzero] = -2.0 * k0[nonzero] * delta_theta[nonzero] / sin_theta[nonzero]
        force0 = (
            coef[:, None]
            * (cos_theta[:, None] * r21 * norm21inv[:, None] - r23 * norm23inv[:, None])
            * norm21inv[:, None]
        )
        force2 = (
            coef[:, None]
            * (cos_theta[:, None] * r23 * norm23inv[:, None] - r21 * norm21inv[:, None])
            * norm23inv[:, None]
        )
        force1 = -(force0 + force2)

    return pot, (force0, force1, force2)


def evaluate_torsion(r12, r23, r34, torsion_params, explicit_forces=True):
    # Calculate dihedral angles from vectors
    crossA = np.cross(r12, r23, axis=1)
    crossB = np.cross(r23, r34, axis=1)
    crossC = np.cross(r23, crossA, axis=1)
    normA = npl.norm(crossA, axis=1)
    normB = npl.norm(crossB, axis=1)
    normC = npl.norm(crossC, axis=1)
    normcrossB = crossB / np.expand_dims(normB,1)
    cosPhi = np.sum(crossA * normcrossB, axis=1) / normA
    sinPhi = np.sum(crossC * normcrossB, axis=1) / normC
    phi = -np.arctan2(sinPhi, cosPhi)

    ntorsions = len(torsion_params[0]["idx"])
    pot = np.zeros(ntorsions, dtype=r12.dtype)
    if explicit_forces:
        coeff = np.zeros(
            ntorsions, dtype=r12.dtype
        )
    for i in range(0, len(torsion_params)):
        idx = torsion_params[i]["idx"]
        k0 = torsion_params[i]["params"][:, 0]
        phi0 = torsion_params[i]["params"][:, 1]
        per = torsion_params[i]["params"][:, 2]

        if np.all(per > 0):  # AMBER torsions
            angleDiff = per * phi[idx] - phi0
            # todo
        

            # pot = np.put_along_axis.at(pot, idx, k0 * (1 + np.cos(angleDiff)))
            # print(idx, "INDEX",  k0 * (1 + np.cos(angleDiff)))
            # pot.scatter_add_(0, idx, k0 * (1 + np.cos(angleDiff)))
            if explicit_forces:
                # todo 
                # coeff = np.put_along_axis.at(coeff, idx, -per * k0 * np.sin(angleDiff))
                pass
                # coeff.scatter_add_(0, idx, -per * k0 * np.sin(angleDiff))
        else:  # CHARMM torsions
            angleDiff = phi[idx] - phi0
            angleDiff[angleDiff < -pi] = angleDiff[angleDiff < -pi] + 2 * pi
            angleDiff[angleDiff > pi] = angleDiff[angleDiff > pi] - 2 * pi

            # todo 
            # pot = np.put_along_axis.at(pot, idx, k0 * angleDiff**2)
            # pot.scatter_add_(0, idx, k0 * angleDiff**2)
            if explicit_forces:
                # coeff = np.put_along_axis.at(coeff, idx, 2 * k0 * angleDiff)
                pass
            #     coeff.scatter_add_(0, idx, 2 * k0 * angleDiff)

    # coeff.unsqueeze_(1)

    force0, force1, force2, force3 = None, None, None, None
    if explicit_forces:
        # Taken from OpenMM
        normDelta2 = npl.norm(r23, axis=1)
        norm2Delta2 = normDelta2**2
        forceFactor0 = (-coeff * normDelta2) / (normA**2)
        forceFactor1 = np.sum(r12 * r23, axis=1) / norm2Delta2
        forceFactor2 = np.sum(r34 * r23, axis=1) / norm2Delta2
        forceFactor3 = (coeff * normDelta2) / (normB**2)

        force0vec = np.expand_dims(forceFactor0,1) * crossA
        force3vec = np.expand_dims(forceFactor3,1) * crossB
        s = (
            np.expand_dims(forceFactor1,1) * force0vec
            - np.expand_dims(forceFactor2,1) * force3vec
        )

        force0 = -force0vec
        force1 = force0vec + s
        force2 = force3vec - s
        force3 = -force3vec

    return pot, (force0, force1, force2, force3)


In [172]:
src = torch.ones((2, 5))
index = torch.tensor([[0, 1, 2, 0, 0]])
foo = torch.zeros(3, 5, dtype=src.dtype).scatter_add_(0, index, src)


srcN = np.zeros((3, 5))
ixs = [0, 1, 2, 0, 0]
indexN = list(zip(ixs, range(len(ixs))))
# print(indexN)
# np.put_along_axis(np.zeros((3, 5)), indexN, srcN, axis=0)

# print(foo)

bar = np.zeros((3,5))
# print(indexN)
# bar[(0,2), (0,3)]+=1
# print(bar)


for i,j in zip(ixs, range(len(ixs))):
    bar[(i,j)]+=1

# bar[[0,1,2], [0,1,0]]+=1

# print(bar)

# np.add.at(srcN, np.array([0,1,2]), bar)
# srcN


array([[[3, 4],
        [1, 2]],

       [[1, 2],
        [3, 4]]])

In [174]:
# from torchmd.forces import Forces
# import torch
forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5, terms=["bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj"])



(1, 3, 3)

In [189]:
nreplicas = 1
natoms = 688
box = np.zeros((nreplicas, 3, 3))

# pos = np.zeros((nreplicas, natoms, 3))
# vel = np.zeros((nreplicas, natoms, 3))
the_forces = np.zeros((nreplicas, natoms, 3))

pos = set_positions(nreplicas, mol.coords)
print(box[0][np.array(np.eye(3), dtype=bool)], "BOX")
box = set_box(nreplicas, mol.box)

Epot = forces.compute(pos, box, the_forces, returnDetails=True)

[0. 0. 0.] BOX
SBOX [19.83880997 19.61930084 19.63419914]
[[-0.15080452 -0.94981194  0.51304436]
 [ 0.31990528 -0.79803658 -0.66999912]
 [ 0.61557102  0.08943748  0.89508247]
 ...
 [ 0.180902    0.46327591  0.81785178]
 [-0.91474628 -0.27151966  0.07581782]
 [-1.09564829 -0.73479557 -0.74203396]] DIR VEC
[[ 8.487608   8.890593  13.186471 ]
 [ 8.638412   9.8404045 12.673427 ]
 [ 8.318507  10.638441  13.343426 ]
 ...
 [ 5.280441  13.81879    2.2057352]
 [ 5.099539  13.355515   1.3878834]
 [ 6.195187  14.09031    2.1299174]] ATOM POS
[[  0   1]
 [  1   2]
 [  1   3]
 ...
 [685 686]
 [685 687]
 [686 687]] ATOM IDX
[1.08999965 1.09000027 1.08999972 1.49404601 1.21000704 1.34128437
 1.00999977 1.53651778 1.08999987 1.52856332 1.5401054  1.08999978
 1.0899997  1.09000042 1.25078345 1.35531032 1.01000023 1.48006959
 1.09000075 1.08999995 1.08999986 0.95720068 0.95719939 1.51360078
 0.95719981 0.95720012 1.51360022 0.9572003  0.95720045 1.5135999
 0.95720046 0.95720012 1.51360019 0.95719959 0.9

In [190]:
Epot

[{'bonds': 3.9577550537286794,
  'angles': 2.84456923922962,
  'dihedrals': 0.0,
  'impropers': 0.0,
  '1-4': 0.0,
  'electrostatics': -2568.4977349687447,
  'lj': 359.2511115243471,
  'external': 0.0}]